In [1]:
from timemachine.potentials import bonded, nonbonded, implicit
import numpy as onp
import jax
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

In [2]:
pdb = PDBFile('hydrogen.pdb')
forcefield = ForceField('hydrogen.xml', 'amber14/tip3p.xml')
system = forcefield.createSystem(pdb.topology)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
#simulation = Simulation(pdb.topology, system, integrator)
#simulation.context.setPositions(pdb.positions)
#state = simulation.context.getState(getForces=True)
context = Context(system, integrator)
context.setPositions(pdb.positions)
state = context.getState(getForces=True, getEnergy=True)
#simulation.minimizeEnergy(maxIterations=100)
#simulation.reporters.append(PDBReporter('output.pdb',10000))
#simulation.reporters.append(StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True))
#simulation.step(10000)
arr = onp.array(state.getForces().value_in_unit(kilojoules_per_mole/nanometer))
arr2 = onp.array(state.getPotentialEnergy().value_in_unit(kilojoules_per_mole))
print(arr)
print(arr2)
print(pdb.positions)

[[ 63.66100188  25.46440075  50.9288015 ]
 [-63.66100188 -25.46440075 -50.9288015 ]]
0.7294901687515774
[Vec3(x=0.05, y=0.03, z=0.0), Vec3(x=0.1, y=0.05, z=0.04000000000000001)] nm


In [3]:
def distance(conf):
    return onp.linalg.norm(conf[0] - conf[1])

In [4]:
newconf = onp.array([[pdb.positions[0][0],pdb.positions[0][1],pdb.positions[0][2]],
                 [pdb.positions[1][0],pdb.positions[1][1],pdb.positions[1][2]]])
print(distance(newconf))

0.0670820393249937 nm


In [5]:
####################################
## METHANE HARMONIC BOND TESTING  ##
####################################

#conf = onp.array([
#            [0.5,0.3,0.0],
#            [1.0, 0.5, 0.4]
#        ])
#params = onp.array([500,0.5])
#bond_idxs = onp.array([[0,1]])
#param_idxs = onp.array([[0,1]])
#dEdx_fn = jax.grad(bonded.harmonic_bond, argnums=(0,))
#timemachine_forces = dEdx_fn(conf,params,None,bond_idxs,param_idxs)[0]
pdb = PDBFile('methane.pdb')
forcefield = ForceField('methane.xml', 'amber14/tip3p.xml')
system = forcefield.createSystem(pdb.topology)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
context = Context(system, integrator)
context.setPositions(pdb.positions)
state = context.getState(getForces=True, getEnergy=True)
openmm_forces = onp.array(state.getForces().value_in_unit(kilojoules_per_mole/nanometer))
#conf = onp.array([[pdb.positions[0][0],pdb.positions[0][1],pdb.positions[0][2]],
#                     [pdb.positions[1][0],pdb.positions[1][1],pdb.positions[1][2]],
#                     [pdb.positions[2][0],pdb.positions[2][1],pdb.positions[2][2]],
#                     [pdb.positions[3][0],pdb.positions[3][1],pdb.positions[3][2]],
#                    [pdb.positions[4][0],pdb.positions[4][1],pdb.positions[4][2]]], dtype=onp.float64)
conf = onp.array([[0.3,-0.4,0.0],
                  [0.3,0.7,0.0],
                  [0.8,-0.7,0.9],
                  [0.8,-0.7,-0.9],
                  [-0.8,-0.7,0.0]])
params = onp.array([500,1.091])
bond_idxs = onp.array([[0,1],
            [0,2],
            [0,3],
            [0,4]])
param_idxs = onp.array([[0,1],
             [0,1],
             [0,1],
             [0,1]])
dEdx_fn = jax.grad(bonded.harmonic_bond, argnums=(0,))
timemachine_forces = dEdx_fn(conf,params,None,bond_idxs,param_idxs)[0]
print(openmm_forces)
print(timemachine_forces)
print(state.getPotentialEnergy())
print(bonded.harmonic_bond(conf,params,None,bond_idxs,param_idxs))
#print(onp.linalg.norm(openmm_forces + timemachine_forces))

/home/gqi/venv/lib/python3.6/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


[[-37.27597516  25.4053906    0.        ]
 [  1.28960678  -4.5          0.        ]
 [  5.15866867 -10.11664772   4.85472242]
 [  5.15866867 -10.11664772  -4.85472242]
 [ 25.66903105  -0.67209515   0.        ]]
[[ 32.402775   -3.2394104   0.       ]
 [  0.          4.5000315   0.       ]
 [ -4.3407025   2.6044214  -7.8132644]
 [ -4.3407025   2.6044214   7.8132644]
 [-23.72137    -6.4694643   0.       ]]
0.16444932260835116 kJ/mol
0.7981491


In [6]:
#####################################
## METHANE HARMONIC ANGLE TESTING  ##
#####################################

pdb = PDBFile('methane.pdb')
forcefield = ForceField('methane.xml', 'amber14/tip3p.xml')
system = forcefield.createSystem(pdb.topology)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
context = Context(system, integrator)
context.setPositions(pdb.positions)
state = context.getState(getForces=True, getEnergy=True)
#for force in system.getForces():
#    print(dir(force))
openmm_forces = onp.array(state.getForces().value_in_unit(kilojoules_per_mole/nanometer))
angleparams = onp.array([75,1.9111355])
angle_idxs = onp.array([[1,0,2],[1,0,3],[1,0,4],[2,0,3],[2,0,4],[3,0,4]])
angleparam_idxs = onp.array([[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]])
dEdx_angles = jax.grad(bonded.harmonic_angle, argnums=(0,))
timemachine_angleforces = dEdx_angles(conf, angleparams, None, angle_idxs, angleparam_idxs, False)[0]
print(openmm_forces)
#print(timemachine_angleforces)
timemachine_total = timemachine_forces + timemachine_angleforces
print(timemachine_total)
print(onp.linalg.norm(openmm_forces + timemachine_total))
for i in range(5):
    for j in range(3):
        if openmm_forces[i][j] + timemachine_total[i][j] > 0.001:
            print("bad")
        

[[-37.27597516  25.4053906    0.        ]
 [  1.28960678  -4.5          0.        ]
 [  5.15866867 -10.11664772   4.85472242]
 [  5.15866867 -10.11664772  -4.85472242]
 [ 25.66903105  -0.67209515   0.        ]]
[[ 37.276035   -25.405466     0.        ]
 [ -1.2896104    4.5000315    0.        ]
 [ -5.1586776   10.116667    -4.8547525 ]
 [ -5.1586776   10.116667     4.8547525 ]
 [-25.66907      0.67210054   0.        ]]
0.00012136947


In [7]:
##################################
## FORMALDEHYDE BONDED TESTING  ##
##################################

pdb = PDBFile('formaldehyde.pdb')
forcefield = ForceField('formaldehyde.xml', 'amber14/tip3p.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=NoCutoff)
#for force in system.getForces():
#    print((force))
print(system.getForces()[3].getNumTorsions())
#print(system.getForces()[0].getNumParticles())
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
context = Context(system, integrator)
context.setPositions(pdb.positions)
state = context.getState(getForces=True, getEnergy=True)
openmm_forces = onp.array(state.getForces().value_in_unit(kilojoules_per_mole/nanometer))
conf = onp.array([[1.1,-0.3,0.0],
                  [2.2,0.2,0.0],
                  [0.2,0.4,0.0],
                  [0.9,-1.4,0.0]])
params = onp.array([47697.6,1.229,28451.2,1.090])
bond_idxs = onp.array([[0,1],
            [0,2],
            [0,3]])
param_idxs = onp.array([[0,1],
             [2,3],
             [2,3]])
dEdx_fn = jax.grad(bonded.harmonic_bond, argnums=(0,))
timemachine_forces = dEdx_fn(conf,params,None,bond_idxs,param_idxs)[0]
angleparams = onp.array([2657.3,2.0943985])
angle_idxs = onp.array([[1,0,2],[1,0,3],[2,0,3]])
angleparam_idxs = onp.array([[0,1],[0,1],[0,1]])
dEdx_angles = jax.grad(bonded.harmonic_angle, argnums=(0,))
timemachine_angleforces = dEdx_angles(conf, angleparams, None, angle_idxs, angleparam_idxs, False)[0]
print(openmm_forces)
#print(timemachine_angleforces)
timemachine_total = timemachine_forces + timemachine_angleforces
print(timemachine_total)
print(onp.linalg.norm(openmm_forces + timemachine_total))
for i in range(4):
    for j in range(3):
        if openmm_forces[i][j] + timemachine_total[i][j] > 0.01:
            print("bad")

1
[[-2576.17958298   -20.10459936   -10.95255043]
 [ 1010.89947434   161.50693073     3.57698901]
 [ 1129.61300108  -872.86704219     3.51367947]
 [  435.66710756   731.46471082     3.86188194]]
[[ 2576.17        20.096802     0.      ]
 [-1010.89636   -161.5054       0.      ]
 [-1129.6108     872.8702       0.      ]
 [ -435.66278   -731.46155      0.      ]]
12.649645
bad
bad
bad


In [56]:
###################################
## FORMALDEHYDE TORSION TESTING  ##
###################################

torsion_params = onp.array([46.024,3.1,2])
torsion_idxs = onp.array([[1,0,2,3]])
torsionparam_idxs = onp.array([[0,1,2]])
dEdx_torsions = jax.grad(bonded.periodic_torsion, argnums=(0,))
timemachine_torsionforces = dEdx_torsions(conf, torsion_params, None, torsion_idxs, torsionparam_idxs)[0]
print(timemachine_torsionforces)
timemachine_total = timemachine_forces + timemachine_angleforces + timemachine_torsionforces
print(onp.linalg.norm(openmm_forces + timemachine_total))

[[ 0.         0.        10.952621 ]
 [ 0.         0.        -3.5770123]
 [ 0.         0.        -3.513702 ]
 [ 0.         0.        -3.8619075]]
0.014545058


In [8]:
########################################
## FORMALDEHYDE TOTAL BONDED TESTING  ##
########################################

pdb = PDBFile('formaldehyde.pdb')
forcefield = ForceField('formaldehyde.xml', 'amber14/tip3p.xml')
system = forcefield.createSystem(pdb.topology)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
context = Context(system, integrator)
context.setPositions(pdb.positions)
state = context.getState(getForces=True, getEnergy=True)
openmm_forces = onp.array(state.getForces().value_in_unit(kilojoules_per_mole/nanometer))
conf = onp.array([[1.1,-0.3,0.0],
                  [2.2,0.2,0.0],
                  [0.2,0.4,0.0],
                  [0.9,-1.4,0.0]])
 #                 [-2.9,0.8,0.0],
 #                 [-2.3,0.0,0.0],
 #                 [-3.0,-0.7,0.0]])

#Harmonic Bond
params = onp.array([47697.6,1.229,28451.2,1.090])
bond_idxs = onp.array([[0,1],
            [0,2],
            [0,3]])
param_idxs = onp.array([[0,1],
             [2,3],
             [2,3]])
dEdx_fn = jax.grad(bonded.harmonic_bond, argnums=(0,))
timemachine_bondforces = dEdx_fn(conf,params,None,bond_idxs,param_idxs)[0]

#Harmonic Angle
angleparams = onp.array([2657.3,2.0943985])
angle_idxs = onp.array([[1,0,2],[1,0,3],[2,0,3]])
angleparam_idxs = onp.array([[0,1],[0,1],[0,1]])
dEdx_angles = jax.grad(bonded.harmonic_angle, argnums=(0,))
timemachine_angleforces = dEdx_angles(conf, angleparams, None, angle_idxs, angleparam_idxs, False)[0]

#Periodic Torsion
torsion_params = onp.array([46.024,3.1,2])
torsion_idxs = onp.array([[1,0,2,3]])
torsionparam_idxs = onp.array([[0,1,2]])
dEdx_torsions = jax.grad(bonded.periodic_torsion, argnums=(0,))
timemachine_torsionforces = dEdx_torsions(conf, torsion_params, None, torsion_idxs, torsionparam_idxs)[0]

print(openmm_forces)
timemachine_total = timemachine_bondforces + timemachine_angleforces + timemachine_torsionforces
print(timemachine_total)
print(onp.linalg.norm(openmm_forces + timemachine_total))

#modeller = Modeller(pdb.topology, pdb.positions)
#modeller.addSolvent(forcefield, numAdded=2)

[[-2576.17958298   -20.10459936   -10.95255043]
 [ 1010.89947434   161.50693073     3.57698901]
 [ 1129.61300108  -872.86704219     3.51367947]
 [  435.66710756   731.46471082     3.86188194]]
[[ 2576.17         20.096802     10.952621 ]
 [-1010.89636    -161.5054       -3.5770123]
 [-1129.6108      872.8702       -3.513702 ]
 [ -435.66278    -731.46155      -3.8619075]]
0.014545058


In [106]:
print(dir(system))

['__class__', '__copy__', '__deepcopy__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__swig_destroy__', '__swig_getmethods__', '__swig_setmethods__', '__weakref__', 'addConstraint', 'addForce', 'addParticle', 'getConstraintParameters', 'getDefaultPeriodicBoxVectors', 'getForce', 'getForces', 'getNumConstraints', 'getNumForces', 'getNumParticles', 'getParticleMass', 'getVirtualSite', 'isVirtualSite', 'removeConstraint', 'removeForce', 'setConstraintParameters', 'setDefaultPeriodicBoxVectors', 'setParticleMass', 'setVirtualSite', 'this', 'usesPeriodicBoundaryConditions']


In [112]:
print(openmm.XmlSerializer.serialize(system))

<?xml version="1.0" ?>
<System openmmVersion="7.4" type="System" version="1">
	<PeriodicBoxVectors>
		<A x="2" y="0" z="0"/>
		<B x="0" y="2" z="0"/>
		<C x="0" y="0" z="2"/>
	</PeriodicBoxVectors>
	<Particles>
		<Particle mass="12"/>
		<Particle mass="16"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
	</Particles>
	<Constraints/>
	<Forces>
		<Force alpha="0" cutoff="1" dispersionCorrection="1" ewaldTolerance=".0005" forceGroup="0" ljAlpha="0" ljnx="0" ljny="0" ljnz="0" method="0" nx="0" ny="0" nz="0" recipForceGroup="-1" rfDielectric="78.3" switchingDistance="-1" type="NonbondedForce" useSwitchingFunction="0" version="3">
			<GlobalParameters/>
			<ParticleOffsets/>
			<ExceptionOffsets/>
			<Particles>
				<Particle eps="0" q="0" sig="0"/>
				<Particle eps="0" q="-0" sig="0"/>
				<Particle eps="0" q="0" sig="0"/>
				<Particle eps="0" q="0" sig="0"/>
			</Particles>
			<Exceptions>
				<Exception eps="0" p1="0" p2="1" q="0" sig="1"/>
				<Exception eps="0" p1="0" p2="2" q

In [2]:
################################
## PROPANE NONBONDED TESTING  ##
################################

pdb = PDBFile('propane.pdb')
forcefield = ForceField('propane.xml', 'amber14/tip3p.xml')
system = forcefield.createSystem(pdb.topology)
#print(system.getForce(0).getEwaldErrorTolerance())
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
context = Context(system, integrator)
context.setPositions(pdb.positions)
state = context.getState(getForces=True, getEnergy=True)
#openmm_forces = onp.array(state.getForces().value_in_unit(kilocalories_per_mole/angstrom))
openmm_forces = onp.array(state.getForces().value_in_unit(kilojoules_per_mole/nanometer))
print(openmm_forces)

#Lennard Jones
conf = onp.array([[-.246,.375,-.005],
                 [.565,-.112,-1.213],
                 [2.013,.392,-1.165],
                 [-0.282,1.488,0.032],
                 [-1.295,0.002,-0.051],
                 [0.201,0.013,0.950],
                 [0.561,-1.229,-1.237],
                 [0.079,0.242,-2.153],
                 [2.591,0.032,-2.047],
                 [2.050,1.506,-1.166],
                 [2.533,0.031,-0.248]])
vdwparams = onp.array([3.50,2.76,2.50,1.26])
#vdwparams=onp.array([0,0,0,0])
vdwparam_idxs = onp.array([[0,1],
                          [0,1],
                          [0,1],
                          [2,3],
                          [2,3],
                          [2,3],
                          [2,3],
                          [2,3],
                          [2,3],
                          [2,3],
                          [2,3]])
scale_matrix = onp.array([[0,0,0,0,0,0,0,0,.5,.5,.5],
               [0,0,0,0,0,0,0,0,0,0,0],
               [0,0,0,.5,.5,.5,0,0,0,0,0],
               [0,0,.5,0,0,0,.5,.5,1,1,1],
               [0,0,.5,0,0,0,.5,.5,1,1,1],
               [0,0,.5,0,0,0,.5,.5,1,1,1],
               [0,0,0,.5,.5,.5,0,0,.5,.5,.5],
               [0,0,0,.5,.5,.5,0,0,.5,.5,.5],
               [.5,0,0,1,1,1,.5,.5,0,0,0],
               [.5,0,0,1,1,1,.5,.5,0,0,0],
               [.5,0,0,1,1,1,.5,.5,0,0,0]])
#print(nonbonded.lennard_jones(conf,vdwparams,None,vdwparam_idxs,scale_matrix))
#print(dir(jax))
dEdx_lj = jax.jacobian(nonbonded.lennard_jones, argnums=(0,))
timemachine_ljforces = onp.sum(dEdx_lj(conf,vdwparams,None,vdwparam_idxs,scale_matrix)[0], axis=0)

#print(timemachine_ljforces)
#print(onp.linalg.norm(openmm_forces + timemachine_ljforces))

#Electrostatics
#conf = conf * 0.1
elecparams = onp.array([-0.200,0.075])
elecparam_idxs = onp.array([0,0,0,1,1,1,1,1,1,1,1])
box = onp.array([[20,0,0],
                [0,20,0],
                [0,0,20]])
scale_matrix_elec = onp.array([[0,0,0,0,0,0,0,0,.833333,.833333,.833333],
               [1,0,0,0,0,0,0,0,0,0,0],
               [0,0,0,.833333,.833333,.833333,0,0,0,0,0],
               [0,0,.833333,0,0,0,.833333,.833333,1,1,1],
               [0,0,.833333,0,0,0,.833333,.833333,1,1,1],
               [0,0,.833333,0,0,0,.833333,.833333,1,1,1],
               [0,0,0,.833333,.833333,.833333,0,0,.833333,.833333,.833333],
               [0,0,0,.833333,.833333,.833333,0,0,.833333,.833333,.833333],
               [.833333,0,0,1,1,1,.833333,.833333,0,0,0],
               [.833333,0,0,1,1,1,.833333,.833333,0,0,0],
               [.833333,0,0,1,1,1,.833333,.833333,0,0,0]])
dEdx_elec = jax.grad(nonbonded.electrostatic, argnums=(0,))
#timemachine_elecforces = dEdx_elec(conf,elecparams,box,elecparam_idxs,scale_matrix_elec,cutoff=10,alpha=.1732,kmax=9)[0]
timemachine_elecforces = dEdx_elec(conf,elecparams,None,elecparam_idxs,scale_matrix_elec)[0]
print(timemachine_elecforces)
#timemachine_nonbonded_total = timemachine_ljforces + timemachine_elecforces
#print(timemachine_nonbonded_total)
#print(onp.linalg.norm(openmm_forces + timemachine_nonbonded_total))
print(onp.linalg.norm(openmm_forces + timemachine_elecforces))

print(state.getPotentialEnergy())
print(nonbonded.electrostatic(conf,elecparams,None,elecparam_idxs,scale_matrix_elec))

[[ 84.4510403   -9.40195761 -59.56474648]
 [  9.97940445  15.55190468  19.63879776]
 [-97.46979275 -10.80207782  33.86793208]
 [-10.11910043   1.66505205  16.89696697]
 [-10.04970725   8.45401545  18.88997263]
 [-12.54517236   9.06197513  12.29245627]
 [ -9.4398458  -22.16476402 -18.69371755]
 [-12.6777435  -12.26634211 -24.85809146]
 [ 21.09589712   8.70344306   2.88414855]
 [ 19.59587083   1.89904513   1.62660045]
 [ 17.17912602   9.29971297  -2.98030849]]


/home/gqi/venv/lib/python3.6/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


[[[0.       0.04     0.04     0.005625 0.005625 0.005625 0.005625
   0.005625 0.005625 0.005625 0.005625]
  [0.04     0.       0.04     0.005625 0.005625 0.005625 0.005625
   0.005625 0.005625 0.005625 0.005625]
  [0.04     0.04     0.       0.005625 0.005625 0.005625 0.005625
   0.005625 0.005625 0.005625 0.005625]
  [0.04     0.04     0.04     0.       0.005625 0.005625 0.005625
   0.005625 0.005625 0.005625 0.005625]
  [0.04     0.04     0.04     0.005625 0.       0.005625 0.005625
   0.005625 0.005625 0.005625 0.005625]
  [0.04     0.04     0.04     0.005625 0.005625 0.       0.005625
   0.005625 0.005625 0.005625 0.005625]
  [0.04     0.04     0.04     0.005625 0.005625 0.005625 0.
   0.005625 0.005625 0.005625 0.005625]
  [0.04     0.04     0.04     0.005625 0.005625 0.005625 0.005625
   0.       0.005625 0.005625 0.005625]
  [0.04     0.04     0.04     0.005625 0.005625 0.005625 0.005625
   0.005625 0.       0.005625 0.005625]
  [0.04     0.04     0.04     0.005625 0.005625 0.00

In [92]:
print(openmm.XmlSerializer.serialize(system))

<?xml version="1.0" ?>
<System openmmVersion="7.4" type="System" version="1">
	<PeriodicBoxVectors>
		<A x="2" y="0" z="0"/>
		<B x="0" y="2" z="0"/>
		<C x="0" y="0" z="2"/>
	</PeriodicBoxVectors>
	<Particles>
		<Particle mass="12.01"/>
		<Particle mass="12.01"/>
		<Particle mass="12.01"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
	</Particles>
	<Constraints/>
	<Forces>
		<Force alpha="0" cutoff="1" dispersionCorrection="1" ewaldTolerance=".0005" forceGroup="0" ljAlpha="0" ljnx="0" ljny="0" ljnz="0" method="3" nx="0" ny="0" nz="0" recipForceGroup="-1" rfDielectric="78.3" switchingDistance="-1" type="NonbondedForce" useSwitchingFunction="0" version="3">
			<GlobalParameters/>
			<ParticleOffsets/>
			<ExceptionOffsets/>
			<Particles>
				<Particle eps="0" q="-.2" sig="0"/>
				<Particle eps="0" q="-.2" sig="0"/>
	

In [3]:
#######################################
## PROPANE IMPLICIT SOLVENT TESTING  ##
#######################################

pdb = PDBFile('propane.pdb')
forcefield = ForceField('propane.xml', 'amber14/tip3p.xml')
system = forcefield.createSystem(pdb.topology, nonbondedCutoff = 2.0*nanometers)
#print(dir(system.getForce(1)))
#print(system.getForce(1).getSolventDielectric())
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
context = Context(system, integrator)
context.setPositions(pdb.positions)
state = context.getState(getForces=True, getEnergy=True)
#openmm_forces = onp.array(state.getForces().value_in_unit(kilocalories_per_mole/angstrom))
openmm_forces = onp.array(state.getForces().value_in_unit(kilojoules_per_mole/nanometer))
print(openmm_forces)

conf = onp.array([[-.246,.375,-.005],
                 [.565,-.112,-1.213],
                 [2.013,.392,-1.165],
                 [-0.282,1.488,0.032],
                 [-1.295,0.002,-0.051],
                 [0.201,0.013,0.950],
                 [0.561,-1.229,-1.237],
                 [0.079,0.242,-2.153],
                 [2.591,0.032,-2.047],
                 [2.050,1.506,-1.166],
                 [2.533,0.031,-0.248]])
#conf = conf*0.1
#print(conf)

gbsaparams = onp.array([-.200,.170,0.79,.075,.115,0.85])
gbsaparam_idxs = onp.array([[0,1,2],
                 [0,1,2],
                 [0,1,2],
                 [3,4,5],
                 [3,4,5],
                 [3,4,5],
                 [3,4,5],
                 [3,4,5],
                 [3,4,5],
                 [3,4,5],
                 [3,4,5]])
dEdx_gbsa = jax.jacobian(implicit.gbsa, argnums=(0,))
timemachine_gbsaforces = onp.sum(dEdx_gbsa(conf,gbsaparams,None,gbsaparam_idxs)[0], axis=0)
print(timemachine_gbsaforces)
print(onp.linalg.norm(openmm_forces + timemachine_gbsaforces))

[[ 33.3825264  -14.18481731 -40.39640427]
 [  9.97940159  15.55190372  19.63879585]
 [-52.09205627 -14.84515381   3.50083256]
 [ -3.89016008  -6.04007626   6.09203243]
 [  3.3837471    5.02896786   6.32891369]
 [ -7.636796     5.38570547  -1.02988434]
 [ -0.94064236   9.62148857  -1.6693902 ]
 [  3.88286591  -5.0775919    7.49241447]
 [  3.10873413   5.03380966   6.46546555]
 [  7.28671455  -5.95027542   0.34764838]
 [  3.53564453   5.47604322  -6.77042007]]


/home/gqi/venv/lib/python3.6/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


[[-23.173428   11.0803385  29.980198 ]
 [-13.48734   -20.91843   -26.537472 ]
 [ 37.74249    11.53939    -1.2840369]
 [  2.8130014 -16.435127   -3.251977 ]
 [ 16.02035     4.1708465  -1.751616 ]
 [ -3.8953629   4.0841374 -16.000898 ]
 [  1.0748172  18.89991     2.3649633]
 [  8.581567   -3.9342117  16.636112 ]
 [-10.805238    3.957204   12.032063 ]
 [ -4.0748277 -16.477299    0.2928971]
 [-10.796021    4.033245  -12.480236 ]]
72.002846


In [115]:
print(openmm.XmlSerializer.serialize(system))

<?xml version="1.0" ?>
<System openmmVersion="7.4" type="System" version="1">
	<PeriodicBoxVectors>
		<A x="2" y="0" z="0"/>
		<B x="0" y="2" z="0"/>
		<C x="0" y="0" z="2"/>
	</PeriodicBoxVectors>
	<Particles>
		<Particle mass="12.01"/>
		<Particle mass="12.01"/>
		<Particle mass="12.01"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
		<Particle mass="1.008"/>
	</Particles>
	<Constraints/>
	<Forces>
		<Force alpha="0" cutoff="1" dispersionCorrection="1" ewaldTolerance=".0005" forceGroup="0" ljAlpha="0" ljnx="0" ljny="0" ljnz="0" method="0" nx="0" ny="0" nz="0" recipForceGroup="-1" rfDielectric="1" switchingDistance="-1" type="NonbondedForce" useSwitchingFunction="0" version="3">
			<GlobalParameters/>
			<ParticleOffsets/>
			<ExceptionOffsets/>
			<Particles>
				<Particle eps="0" q="-0" sig="0"/>
				<Particle eps="0" q="-0" sig="0"/>
				<P

In [32]:
#TORSION TEST CASE

conf = onp.array(
            [[-0.6000563454193615, 0.376172954382274 ,-0.2487295756125901],
             [ 0.561317027011325 , 0.2066950040043141, 0.3670430960815993],
             [-1.187055522272264 ,-0.3415864358441354, 0.0871382207830652],
             [ 0.9399773448903637,-0.6888774474110431, 0.2104211949995816]], dtype=onp.float64)

nan_conformers = onp.array([
            [[-0.6000563454193615, 0.376172954382274 ,-0.2487295756125901],
             [ 0.5613170270113252, 0.2066950040043142, 0.3670430960815993],
             [-1.187055522272264 ,-0.3415864358441354, 0.0871382207830652],
             [ 1.2278668040866427, 0.8805184219394547, 0.099391329616366 ]],
            [[-0.6000563454193615, 0.376172954382274 ,-0.2487295756125901],
             [ 0.561317027011325 , 0.206695004004314 , 0.3670430960815994],
             [-1.187055522272264 ,-0.3415864358441354, 0.0871382207830652],
             [ 0.5494071252089705,-0.5626592973923106, 0.9817919758125693]],
            ], dtype=onp.float64)

torsion_idxs = onp.array([
            [0, 1, 2, 3],
            [0, 1, 2, 3],
            [0, 1, 2, 3],
        ], dtype=onp.int32)

params = onp.array([
            2.3, # k0
            5.4, # k1
            9.0, # k2
            0.0, # t0
            3.0, # t1
            5.8, # t2
            1.0, # n0
            2.0, # n1
            3.0  # n2
        ])

param_idxs = onp.array([
            [0, 3, 6],
            [1, 4, 7],
            [2, 5, 8]
        ], dtype=onp.int32)

box = onp.array([
            [2.0, 0.5, 0.6],
            [0.6, 1.6, 0.3],
            [0.4, 0.7, 1.1]
        ], dtype=onp.float64)
        
print(bonded.periodic_torsion(conf, params, None, torsion_idxs, param_idxs))
dEdx = jax.grad(bonded.periodic_torsion, argnums=(0,))
print(dEdx(conf, params, box, torsion_idxs, param_idxs)[0])

pdb = PDBFile('example.pdb')
forcefield = ForceField('example.xml')
system = forcefield.createSystem(pdb.topology)

26.169462
[[  5.0214257  -9.800515  -12.167998 ]
 [ -3.0019753   1.3290389  16.147926 ]
 [ -3.0457368   6.106178    7.0637503]
 [  1.0262862   2.3652978 -11.043678 ]]


ValueError: No template found for residue 1 (UNK).  This might mean your input topology is missing some atoms or bonds, or possibly that you are using the wrong force field.